# Lorentz transformation
In this lesson we will write and use functions for working with 4-vectors and Lorentz transformations.

## Pre-requisites
First we need to load the modules we need. We can also check whether loading was succeful and the version of the imported module.

In [11]:
import astropy
astropy.__version__

'4.2.1'

numpy is typically imported as np:

In [12]:
import numpy as np
np.__version__

'1.20.1'

We will use matplotlib for plotting:

In [13]:
import matplotlib
matplotlib.__version__

'3.3.4'

From the astropy module we need only the units and the constants, we will import them as u and c.

In [14]:
from astropy import units as u
from astropy import constants as c

We will now import the Abstract Base Class (abc) module, which will allow us to define a general 4-vector class followed by 4-momentum and 4-position subclasses.

In [15]:
import abc

Let's import both the math module and the complex math module.

In [16]:
import cmath
import math

## 4-vectors

Let's first define the Minkowski metric. We will take the time component as the 0th component with positive normalisation.

In [17]:
METRIC = np.diag([1, -1, -1, -1])  # This is a diagonal 4x4 matrix.

Let us now define the 4-vector class. The first method describes how to initialize the 4-vector.

In [18]:
class Vector4:
    """
    Representation of a Lorentz 4-vector.
    """
    def __init__(self, *x):
        """
        Creates a new 4-vector. The arguments can be either the four
        components or another 4-vector.

        >>> Vector4(60, 2, 3, 4)
        Vector4(60, 2, 3, 4)

        >>> vec = Vector4(60, 1, 2, 3)
        >>> Vector4(vec)
        Vector4(60, 1, 2, 3)
        """
        if len(x) == 1 and isinstance(x[0], Vector4):
            x = x[0]
            self._values = np.array(x.components)
        elif len(x) == 4:
            self._values = np.array(list(x))
        else:
            raise TypeError("4-vectors expects four values")

The next two methods define ways to display and to access the internal array of the 4-vector.

In [19]:
    def __repr__(self):
        """
        Returns a string representation of the object.
        """
        if self._values.ndim == 1:
            pattern = "%g"
        else:
            pattern = "%r"

        return "%s(%s)" % (self.__class__.__name__,
                           ", ".join([pattern % _ for _ in self._values]))

    @property
    def components(self):
        """
        Returns the interal array of all components.
        """
        return self._values

Definitions of mathematical operations.

In [20]:
    def __add__(self, other):
        """
        Addition of two 4-vectors. Can only add vectors of the same type (or
        subtype).

        >>> a = Vector4(1, 2, 3, 4)
        >>> b = Vector4(2, 4, 8, 16)
        >>> a + b
        Vector4(3, 6, 11, 20)
        """
        vector = self.__class__(self)
        vector += other
        return vector

    def __iadd__(self, other):
        """
        In-place addition of two 4-vectors.

        >>> a = Vector4(1, 2, 3, 4)
        >>> b = Vector4(2, 4, 8, 16)
        >>> b += a
        >>> b
        Vector4(3, 6, 11, 20)
        """
        self._values = self.components + Vector4(*other).components
        return self

    def __sub__(self, other):
        """
        Subtraction of two 4-vectors.

        >>> a = Vector4(1, 2, 3, 4)
        >>> a - b
        Vector4(-3, -1, 1, 3)
        """
        vector = self.__class__(self)
        vector -= other
        return vector

    def __isub__(self, other):
        """
        In-place subtraction of two 4-vectors.

        >>> a = Vector4(1, 2, 3, 4)
        >>> b = Vector4(4, 3, 2, 1)
        >>> b -= a
        >>> b
        Vector4(-3, -1, 1, 3)
        """
        self._values = self.components - Vector4(*other).components
        return self

    def __mul__(self, other):
        """
        Multiplication of a 4-vector by a scalar or the dot product with
        another 4-vector.
        """
        if hasattr(other, "__len__") and len(other) == 4:
            # Dot product
            other = other.components
            components = self.components

            is_scalar = (other.ndim == 1) and (components.ndim == 1)

            if other.ndim == 1:
                other = other.reshape(other.shape[0], 1)
            if components.ndim == 1:
                components = components.reshape(components.shape[0], 1)

            dot_product = (METRIC.dot(other) * components).sum(axis=0)
            dot_product = dot_product.reshape(dot_product.size)

            if is_scalar:
                return dot_product[0]

            return dot_product

        vector = self.__class__(self)
        vector *= other
        return vector


    def __rmul__(self, other):
        """
        Multiplication of a 4-vector by a scalar from the left.
        """
        return self * other

    def __imul__(self, other):
        """
        Multiplication of a 4-vector by a scalar from the left.
        """
        if hasattr(other, "__len__") and len(other) != 1:
            raise TypeError("In-place multiplication only possible for "
                            "scalars.")

        self._values *= other
        return self

    def __neg__(self):
        """
        Negate all components, equivalent to v * (-1)
        """
        return (-1) * self

    def __truediv__(self, other):
        """
        Division of a 4-vector by a scalar from the left.
        """
        vector = self.__class__(self)
        vector /= other
        return vector

    def __floordiv__(self, other):
        """
        Division of a 4-vector by a scalar from the left.
        """
        vector = self.__class__(self)
        vector //= other
        return vector

    def __ifloordiv__(self, other):
        """
        In-place division of a 4-vector by a scalar.
        """
        if hasattr(other, "__len__") and len(other) != 1:
            raise TypeError("Division only possible by scalars.")

        # in-place div no always possible for numpy arrays depending on their
        # type, i.e. cannot in-place convert int array to float
        self._values = self._values // other
        return self

    def __itruediv__(self, other):
        """
        In-place division of a 4-vector by a scalar.
        """
        if hasattr(other, "__len__") and len(other) != 1:
            raise TypeError("Division only possible by scalars.")

        # in-place div no always possible for numpy arrays depending on their
        # type, i.e. cannot in-place convert int array to float
        self._values = self._values / other
        return self

    def __div__(self, other):
        """
        Legacy support
        """
        return self.__truediv__(other)

    def __idiv__(self, other):
        """
        Legacy support
        """
        return self.__itruediv__(other)


Access to length and components of the 4-vector

In [36]:
    def __len__(self):
        """
        Returns 4, i.e. the length of the vector.
        """
        return 4

    def __iter__(self):
        """
        Returns an iterator over all its values.
        """
        return iter(self._values)

    def __getitem__(self, i):
        """
        Access to the components of the vector.
        """
        return self._values[i]

The magnitude and square of the magnitude.

In [37]:
    @property
    def mag(self):
        """
        Magnitude of vector.
        """
        mag2 = np.asarray(self.mag2, dtype='complex')
        mag = np.sqrt(mag2)
        if (mag.imag == 0).all():
            return mag.real

        return mag

    @property
    def mag2(self):
        """
        Square of magnitude of vector, i.e. the dot product with itself.
        """
        return self * self

The next method performs the Lorentz transformation for a general velocity vector without rotation. We will check to see that it gives the same result as the velocity addition formula when it is along the x-axis.

In [42]:
    # pylint: disable=too-many-arguments
    def boost(self, x, y, z, beta=None, gamma=None):
        """
        Boosts the 4-vector in the direction given by (x, y, z). The magnitude
        of (x, y, z) is ignored. Exactly one of beta and gamma must be given.

        The method returns the transformed 4-vector as measured in the frame
        moving in (x, y, z) direction with velocity defined by beta or gamma.
        This is the opposite of TLorentzVector::Boost().
        """
        x = np.asarray(x)
        y = np.asarray(y)
        z = np.asarray(z)

        if beta is None:
            if gamma is None:
                raise ValueError("beta and gamma cannot be both None")
            gamma = np.asarray(gamma)
            beta = np.sqrt(1 - 1 / gamma**2)
        else:
            if gamma is not None:
                raise ValueError("beta and gamma cannot be both not None")
            beta = np.asarray(beta)
            gamma = 1 / np.sqrt(1 - beta**2)

        shapes = [x.shape,
                  beta.shape,
                  gamma.shape,
                  self.components[0].shape]

        if not any(shapes):
            lift_shape = []
        else:
            lengths = [s[0] for s in shapes if s and s[0] != 1]
            if len(lengths) == 0:
                dim_lift = 1
            else:
                lengths = set(lengths)
                if len(lengths) != 1:
                    raise ValueError("Incompatible boost dims: %s" % lengths)
                dim_lift = lengths.pop()

            lift_shape = [dim_lift]

        dim_lift = np.ones(lift_shape)

        x = x * dim_lift
        y = y * dim_lift
        z = z * dim_lift

        beta = beta * dim_lift
        gamma = gamma * dim_lift

        p = np.stack([x, y, z])
        p = p / np.sqrt((p * p).sum(axis=0))

        A = np.zeros([4, 4] + lift_shape)
        A[1:, 1:, ] = p

        A = (gamma - 1) * (A * A.swapaxes(0, 1))
        A += np.tensordot(np.diag([0, 1, 1, 1]), dim_lift, axes=0)

        bp = -beta * p

        B = np.zeros([4, 4] + lift_shape)
        B[0, 1:, ] = bp
        B[1:, 0, ] = bp
        B[0, 0, ] = 1

        B = gamma * B

        L = B + A

        return self.__class__(*np.einsum('ij...,j...->i...', L, self.components))


The next method uses the previous method to perform the Lorentz transformation when the relative velocity is given by the 4-momentum of a particle.

In [43]:
    def boost_particle(self, momentum):
        """
        Performs a Lorentz transformation of the 4-vector. The boost is
        specified by the 4-momentum of a particle (measured in the laboratory
        frame). The transformation is from the particle rest frame into the
        laboratory frame.

        The method returns the transformed 4-vector.
        """
        return self.boost(*(-momentum.components)[1:],
                          gamma=momentum.e/momentum.m)


Now let's define the 4-momentum as an instance of 4-vectors, with some special methods for initializing and getting special values (e.g. mass, 3-momentum etc.)

In [44]:
class Momentum4(Vector4):
    """
    Representation of 4-momenta.
    """
    @staticmethod
    def e_eta_phi_pt(e, eta, phi, p_t):
        """
        Creates a new energy-momentum vector based on energy, pseudo-rapidity,
        polar angle and transverse momentum.
        """
        p_x = np.cos(phi) * p_t
        p_y = np.sin(phi) * p_t
        p_z = p_t / (np.tan(2*np.arctan(np.exp(-eta))))
        return Momentum4(e, p_x, p_y, p_z)

    @staticmethod
    def m_eta_phi_pt(m, eta, phi, p_t):
        """
        Creates a new energy-momentum vector based on mass, pseudo-rapidity,
        polar angle and transverse momentum.
        """
        p_x = np.cos(phi) * p_t
        p_y = np.sin(phi) * p_t
        p_z = p_t / (np.tan(2*np.arctan(np.exp(-eta))))
        e = np.sqrt(m**2 + p_x**2 + p_y**2 + p_z**2)
        return Momentum4(e, p_x, p_y, p_z)

    @staticmethod
    def m_eta_phi_p(m, eta, phi, p):
        """
        Creates a new energy-momentum vector based on mass, pseudo-rapidity,
        polar angle and momentum.
        """
        theta = 2 * np.arctan(np.exp(-eta))
        p_x = np.cos(phi) * np.sin(theta) * p
        p_y = np.sin(phi) * np.sin(theta) * p
        p_z = np.cos(theta) * p
        e = np.sqrt(m**2 + p_x**2 + p_y**2 + p_z**2)
        return Momentum4(e, p_x, p_y, p_z)

    @staticmethod
    def e_eta_phi_p(e, eta, phi, p):
        """
        Creates a new energy-momentum vector based on energy, pseudo-rapidity,
        polar angle and momentum.
        """
        theta = 2 * np.arctan(np.exp(-eta))
        p_x = np.cos(phi) * np.sin(theta) * p
        p_y = np.sin(phi) * np.sin(theta) * p
        p_z = np.cos(theta) * p
        return Momentum4(e, p_x, p_y, p_z)

    @staticmethod
    def e_m_eta_phi(e, m, eta, phi):
        """
        Creates a new energy-momentum vector based on energy, mass,
        pseudo-rapidity and polar angle.
        """
        p = np.sqrt(e**2 - m**2)
        return Momentum4.e_eta_phi_p(e, eta, phi, p)

    @property
    def m(self):
        """
        Returns the mass.
        """
        return self.mag

    @property
    def m2(self):
        """
        Returns the mass squared.
        """
        return self.mag2

    @property
    def p(self):
        """
        Returns the magnitude of the 3-momentum.
        """
        return np.sqrt(self.p2)

    @property
    def p2(self):
        """
        Returns the square of the magnitude of the 3-momentum.
        """
        return self.p_x**2 + self.p_y**2 + self.p_z**2

    @property
    def p_x(self):
        """
        Returns the x-component of the momentum.
        """
        return self[1]

    @property
    def p_y(self):
        """
        Returns the y-component of the momentum.
        """
        return self[2]

    @property
    def p_z(self):
        """
        Returns the y-component of the momentum.
        """
        return self[3]

    @property
    def e(self):
        """
        Returns the energy of the vector.
        """
        return self[0]


Let's do the same for the 4-position.

In [45]:
class Position4(Vector4):
    """
    Representation of points in space-time.
    """

    @property
    def x(self):
        """
        Returns the x component of the vector.
        """
        return self[1]

    @property
    def y(self):
        """
        Returns the y component of the vector.
        """
        return self[2]

    @property
    def z(self):
        """
        Returns the z component of the vector.
        """
        return self[3]

    @property
    def t(self):
        """
        Returns the t component of the vector.
        """
        return self[0]


### Time dilation
Let's use the 4-position vectors to examine time dilation.

In [ ]:
distance = 1 * u.m
beta = 0.9

Delta_t = 2 * c.c * distance

Event0 = Position4(0,0,0,0)
Event1 = Position4(0.5 * Delta_t, 0, distance, 0)
Event2 = Position4(Delta_t, 0, 0, 0)

Event0_prime = Event0.boost(1,0,0,beta)
Event0_prime
Event1_prime = Event1.boost(1,0,0,beta)
Event1_prime
Event2_prime = Event2.boost(1,0,0,beta)
Event2_prime

Delta_t_prime = Event2_prime[0] - Event1_prime[0]
Delta_t_prime
Delta_t_prime / Delta_t


## Particle Distributions (photons, electrons, protons)
In this course we will not discuss photons of a single energy, but particle distributions of photons, electrons or protons. We will study the particles over several orders of magnitude in energy, let's say from 1 MeV (10⁶ eV) to 100 TeV (10¹⁴ eV). We will create a log space with 15 data points:

We study non-thermal emission, so our photons will not follow a block-body spectrum but typically a power law. A power law has the following form:
$$ F(E) = A \times \left( \frac{E}{E_0} \right)^{-\Gamma} $$
$A$ is the amplitude, i.e. the function's value at the reference energy $E_0$. $\Gamma$ is called the spectral index. Note the minus sign in the exponent, so $\Gamma$ is typically positive. We often use 1 TeV as reference energy:

Let's define some example parameters:

A power law is already defined in astropy. Let's make use of it.

Now we can call this function on our energy array defined above:

### Plotting
We will use pyplot from matplotlib to do some plotting. pyplot is usually imported as plt:

Let's make a plot of our y value over our energy array:

This looks a bit strange. Keep in mind that our energies span 14 orders of magnitude. Let's try to get the energy axis in log scale:

Looks better already. But is our function really zero above an energy of 10^7 eV? Let's try a log-log plot:

Looks good. We see that a power law is represented as a straight line in a log-log plot. Now let's add titles to the axes and a legend:

You see that the y axis is in units TeV⁻¹. It is a differential particle number, how many particles are found in an infinitesimal small bin of with $dE$:
$$\frac{dN}{dE}(E).$$
If you want to know how many particles are in the entire particle spectrum then you have to integrate over it:
$$ N = \int_{E_1}^{E_2} \frac{dN}{dE}(E) dE.$$
For a power law it is easy to do the integration analytically. But it may be difficult for more complicated spectra. We can use numpy for a numerical integration.

There are no units as this is simply a particle number. Keep in mind that the trapezoidal rule is not very accurate and depends on the number of points. Having more points would increase the accuracy. Other, more sophisticated integration modules exist for python.

Similarly we can calculate the total energy in the spectrum. We need to integrate over all energies:
$$ E_{tot} = \int_{E_1}^{E_2} E \frac{dN}{dE}(E) dE.$$
We can achieve this by multiplying the $y$ array with the energies array and then integrate.

### Exercise
Make a plot of three power laws with indices 1.5, 2.0 and 2.5. Keep the amplitude as above. Do not forget the title on the x-axis and the legend. Calculate the number of particles and the total energy in each spectrum.

**[3 marks]**

## Cyclotron Frequency

Here are some values fo an example. We will use the geo-magnetic field:

The electron has one elementary charge, we do not care about the sign here:

Let's put everything together:

Depending on the system you are using the elementary charge has different values and units. So we have to be specific here. All calculations in this course are in standard units, so let's try this.

How many cycles per second? We have to devide by the angle of one full circle, $2\pi$.

### Exercise

We will need the cyclotron frequency more often. Please write a function which takes the magnetic field as first parameter, the "atomic number" of the particle (-1 for an electron, +1 for a proton, +2 for a Helium nucleus and so on) as 2nd parameter with -1 as default, and the mass as third parameter with the electron mass as default. Return the cyclotron frequency in Hz. Make sure that it is never negative!

**[3 marks]**

Let's test it:

## Submission

Before you submit your work you should make a few checks that everything works fine.

1. Save your notebook as a PDF (File->Download As->PDF). This document will help you debugging in the next step.
1. If PDF export does not work: You can do File->Print Preview and then print to a file.
1. Restart the kernel and rerun the entire notebook (Kernel->Restart & Run All). This will delete all variables (but not your code) and rerun the notebook in one go. If this does not go through the endthen you have to fix it. You will see at which cell the run stopped. A common mistake is using a variable that is defined only at a later stage.
1. You think you fixed everything? Redo step 2 (Kernel->Restart & Run All)

You have to download and submit 2 files, the jupyter notebook and a pdf.
- Jupyter notebook. File->Download As->Notebook (.ipynb). Save this file on your disk.
- PDF file. File->Download As->PDF. Save this file on your disk.
- If PDF export does not work. You can do File->Print Preview and then print to a file.

Please submit the two files on Ulwazi.